# 如何让agent直接返回工具结果

典型的 ReAct 循环遵循用户 -> 助手 -> 工具 -> 助手 ..., ->
用户。在某些情况下，您不需要在工具完成后调用LLM，
用户可以直接自己查看结果。

在此示例中，我们将构建一个对话式 ReAct 代理，法学硕士可以在其中
可选择决定返回工具调用的结果作为最终答案。这
当您拥有有时可以生成响应的工具时非常有用
可以接受作为最终答案，并且您希望使用法学硕士来确定
当这种情况时

## 设置

首先我们需要安装所需的软件包：
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core
```
接下来，我们需要为 OpenAI（我们将使用的 LLM）设置 API 密钥。可选地，我们
可以为[LangSmith追踪](https://smith.langchain.com/)设置API密钥，
将为我们提供一流的可观察性。


In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__..."
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Direct Return: LangGraphJS";

Direct Return: LangGraphJS


## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
使用简单的占位符“搜索引擎”。然而，创建起来确实很容易
您自己的工具 - 请参阅文档
[此处](https://js.langchain.com/docs/modules/agents/tools/dynamic)了解如何操作
那。

要添加“return_direct”选项，我们将创建一个自定义 zod 架构来使用
**而不是**工具自动推断的模式。


In [2]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const SearchTool = z.object({
  query: z.string().describe("query to look up online"),
  // **重要** 我们在这里添加一个 **额外** 字段
  // 该工具不直接使用它 - 它由我们的
  // 来决定是否返回
  // 结果直接呈现给用户
  return_direct: z.boolean()
    .describe(
      "Whether or not the result of this should be returned directly to the user without you seeing what it is",
    )
    .default(false),
});

const searchTool = new DynamicStructuredTool({
  name: "search",
  description: "Call to surf the web.",
  // 我们在这里重写默认模式
  // 添加额外字段
  schema: SearchTool,
  func: async ({}: { query: string }) => {
    // 这是实际实现的占位符
    // 不过不要让LLM知道这一点😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈.";
  },
});

const tools = [searchTool];

我们现在可以将这些工具包装在 `ToolNode` 中。
这是一个预先构建的节点，它接收 LangChain 聊天模型生成的工具调用并调用该工具，
返回输出。

In [3]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);

## 设置模型

现在我们需要加载我们想要使用的聊天模型。\
重要的是，这应该满足两个标准：

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2.应该支持
[工具调用](https://js.langchain.com/docs/concepts/tool_calling/)。

注意：这些模型要求不是使用 LangGraph 的要求 - 它们
只是这个例子的要求。


In [4]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  temperature: 0,
  model: "gpt-3.5-turbo",
});
// 这会将工具格式化为模型 API 的 json 架构。
// 然后模型将其用作系统提示符。
const boundModel = model.bindTools(tools);

## 定义代理状态

`langgraph` 中图形的主要类型是
[StateGraph](/langgraphjs/reference/classes/langgraph.StateGraph.html)。

该图由一个状态对象参数化，该状态对象传递给每个
节点。然后每个节点返回操作来更新该状态。这些操作
可以设置状态的特定属性（例如覆盖现有的
值）或添加到现有属性。是否设置或添加表示在
您构建图表所用的状态对象。

对于这个例子，我们将跟踪的状态只是一个消息列表。我们
希望每个节点只将消息添加到该列表中。因此，我们将定义
声明如下：

In [5]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 定义节点

我们现在需要在图中定义一些不同的节点。在`langgraph`中，一个节点
可以是一个函数或一个
[可运行](https://js.langchain.com/docs/expression_language/)。有两个
为此我们需要的主要节点：

1. 代理：负责决定采取什么（如果有）行动。
2. 调用工具的函数：如果代理决定采取行动，该节点
然后将执行该操作。

我们还需要定义一些边。其中一些边缘可能是有条件的。
它们是有条件的原因是基于节点的输出，其中之一
可以采取多条路径。直到该节点才知道所采取的路径
运行（LLM 决定）。

1. 条件边缘：调用代理后，我们应该：如果
代理说要采取行动，那么调用工具的函数应该是
称为 b.如果代理说已经完成，那么就应该完成
2. 正常边缘：调用工具后，应始终返回到
代理决定下一步做什么

让我们定义节点以及一个函数来决定什么条件
边取。


In [6]:
import { RunnableConfig } from "@langchain/core/runnables";
import { END } from "@langchain/langgraph";
import { AIMessage } from "@langchain/core/messages";

// 定义判断是否继续的函数
const shouldContinue = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有函数调用，那么我们就完成了
  if (!lastMessage?.tool_calls?.length) {
    return END;
  } // Otherwise if there is, we check if it's suppose to return direct
  else {
    const args = lastMessage.tool_calls[0].args;
    if (args?.return_direct) {
      return "final";
    } else {
      return "tools";
    }
  }
};

// 定义调用模型的函数
const callModel = async (state: typeof AgentState.State, config?: RunnableConfig) => {
  const messages = state.messages;
  const response = await boundModel.invoke(messages, config);
  // 我们返回一个对象，因为这将被添加到现有列表中
  return { messages: [response] };
};

## 定义图表

我们现在可以将它们放在一起并定义图表！


In [7]:
import { START, StateGraph } from "@langchain/langgraph";

// 定义一个新图
const workflow = new StateGraph(AgentState)
  // 定义我们将在其之间循环的两个节点
  .addNode("agent", callModel)
  // 请注意，“操作”和“最终”节点是相同的！
  .addNode("tools", toolNode)
  .addNode("final", toolNode)
  // 将入口点设置为“agent”
  .addEdge(START, "agent")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“代理”。
    "agent",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue,
  )
  // 我们现在添加从“tools”到“agent”的正常边缘。
  .addEdge("tools", "agent")
  .addEdge("final", END);

// 最后，我们编译它！
const app = workflow.compile();

## 使用它！

我们现在可以使用它了！现在这暴露了
[相同的界面](https://js.langchain.com/docs/expression_language/) 与所有
其他 LangChain 运行程序。


In [8]:
import { HumanMessage, isAIMessage } from "@langchain/core/messages";

const prettyPrint = (message: BaseMessage) => {
  let txt = `[${message._getType()}]: ${message.content}`;
  if (
    isAIMessage(message) && (message as AIMessage)?.tool_calls?.length || 0 > 0
  ) {
    const tool_calls = (message as AIMessage)?.tool_calls
      ?.map((tc) => `- ${tc.name}(${JSON.stringify(tc.args)})`)
      .join("\n");
    txt += ` \nTools: \n${tool_calls}`;
  }
  console.log(txt);
};

const inputs = { messages: [new HumanMessage("what is the weather in sf")] };
for await (const output of await app.stream(inputs, { streamMode: "values" })) {
  const lastMessage = output.messages[output.messages.length - 1];
  prettyPrint(lastMessage);
  console.log("-----\n");
}

[human]: what is the weather in sf
-----

[ai]:  
Tools: 
- search({"query":"weather in San Francisco"})
-----

[tool]: It's sunny in San Francisco, but you better look out if you're a Gemini 😈.
-----

[ai]: The weather in San Francisco is sunny.
-----



In [9]:
const inputs2 = {
  messages: [
    new HumanMessage(
      "what is the weather in sf? return this result directly by setting return_direct = True",
    ),
  ],
};
for await (
  const output of await app.stream(inputs2, { streamMode: "values" })
) {
  const lastMessage = output.messages[output.messages.length - 1];
  prettyPrint(lastMessage);
  console.log("-----\n");
}

[human]: what is the weather in sf? return this result directly by setting return_direct = True
-----

[ai]:  
Tools: 
- search({"query":"weather in San Francisco","return_direct":true})
-----

[tool]: It's sunny in San Francisco, but you better look out if you're a Gemini 😈.
-----



完毕！运行 `tools` 节点后图表 **停止**！
```
```